In [2]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bag_reader.bag_reader import READER
from bag_processor.bag_processor import DepthProcessor


bag_read = READER('/home/nimibot/catkin_ws/src/ros_env_prediction/env_recorder_pkg/bag/2022-10-27-12-42-56.bag')
bag_read.read()
bag_read.depth_df

dp = DepthProcessor()
feat_df = dp.get_features_df(bag_read.depth_df)

class AlgoRunner:
    def __init__(self,bag_obj):
        self.bag_obj = bag_obj
        
        pass

    def __len__(self):
        return len(self.bag_obj.depth_df)

    def get_current_step(self, step):
        data = {}
        data["depth"] = cv2.imread(bag_read.depth_df.frame_path[step])
        data["rgb"] = cv2.imread(bag_read.rgb_df.frame_path[step])
        
        return data

    def detect_stairs_rgb(self,img_rgb):

        gray = cv2.cvtColor(img_rgb,cv2.COLOR_BGR2GRAY)
        blured = cv2.GaussianBlur(img_rgb,(5,5),0,0)
        edges = cv2.Canny(blured,20,150,apertureSize = 3)
        #lines = cv2.HoughLines(edges,1,np.pi/180,200)
        minLineLength = 100
        maxLineGap = 10
        lines = cv2.HoughLinesP(edges,1,np.pi/180,20,minLineLength,maxLineGap)
        return lines


    def run(self):
        
        
        algo_buffer = []
        
        for step in range(len(self)):
            out_data = {}

            in_data = self.get_current_step(step)
            img_grid = dp.split_to_regions(in_data["depth"])
            mean_grid = dp.get_regions_mean(img_grid)
            std_grid = dp.get_regions_std(img_grid)
            lines = self.detect_stairs_rgb(in_data["rgb"])
            out_data["lines"], out_data["mean"], out_data["std"] = lines, mean_grid, std_grid

            self.vis_step(in_data,out_data)
            algo_buffer.append(out_data)
        
        self.save_runner(algo_buffer)

    def save_runner(self,algo_buffer):
        pass

    def vis_step(self,in_data,out_data):
        if "lines" in out_data:
            for line in out_data["lines"]:
                for x1,y1,x2,y2 in line:
                    m = (y1-y2)/(x1-x2)
                    
                    if np.rad2deg(np.arctan(m))<15 and np.rad2deg(np.arctan(m))>-15 : 
                        #print(np.rad2deg(np.arctan(m)))
                        cv2.line(in_data["rgb"],(x1,y1),(x2,y2),(0,255,0),2)

            cv2.imshow("depth", in_data["depth"])
            cv2.imshow("rgb", in_data["rgb"])
            cv2.waitKey(0)
        pass

algo_runner = AlgoRunner(bag_read)


[INFO]  Data folder /home/nimibot/catkin_ws/src/ros_env_prediction/env_recorder_pkg/bag/2022-10-27-12-42-56 already exists. Not creating.


In [3]:
algo_runner.run()

/home/nimibot/py3.7_ws/py3.7_venv/lib/python3.7/site-packages/ipykernel_launcher.py:71: RuntimeWarning: divide by zero encountered in int_scalars


KeyError: 51